In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV, LassoLarsCV
import warnings
import sys
import os
import pickle
# ignoramos os resultados NaN das funções pois vamnos zera-los
np.seterr(invalid='ignore')
# nao quero warning de convergência
warnings.filterwarnings('ignore')

In [3]:
def getResults(fname):
    dataset = []
    algoritmo = []
    msre_l = []
    if os.path.exists(fname):
        fw = open(fname, 'rb')
        dataset, algoritmo, msre_l  = pickle.load(fw)
        fw.close()
    return dataset, algoritmo, msre_l

In [4]:
def storeResults(dataset, algoritmo, msre_l, fname):
    fw = open(fname, 'wb')
    pickle.dump((dataset, algoritmo, msre_l), fw)
    fw.close()

In [5]:
def msre(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [6]:
def geraRede(X, n_inter, inter_min=0, inter_max=3):
    n_inputs = X.shape[1]
    rede = np.random.randint(inter_min, inter_max, size=(n_inputs, n_inter))
    for i in range(0, n_inputs):
        if 0 in X[:,i]:
            rede[i,:] = np.absolute(rede[i,:])
    return rede

In [7]:
def transformData(X, rede):
    n_rows = X.shape[0]
    n_inter = rede.shape[1]
    layers = np.ndarray((n_rows, 3*n_inter))
    for i in range(0, n_inter*3, 3):
        power = X**rede[:,int(i/3)]
        layers[:,i] = np.prod(power,axis=1) # id
        layers[:,i+1] = np.cos(layers[:,i]) # cos
        layers[:,i+2] = np.sqrt(layers[:,i]) # sqrt
    cols = np.any(np.isnan(layers), axis=0)
    layers[:, cols] = 0
    cols = np.any(np.isinf(layers), axis=0)
    layers[:, cols] = 0
    return layers

In [8]:
def fit(X, y, algoritmo, n_inter, inter_min=0, inter_max=3):
    n_inputs = X.shape[1]
    rede = geraRede(X, n_inter, inter_min, inter_max)
    X_transf = transformData(X, rede)
    if algoritmo == 'lasso':
        modelo = LassoCV(max_iter=5e4, cv=3)
    elif algoritmo == 'lassoLars':
        modelo = LassoLarsCV(max_iter=5e4, cv=3)
    modelo.fit(X_transf, y)
    return rede, modelo

In [9]:
def predict(X, rede, modelo):
    X_transf = transformData(X, rede)
    y_hat = modelo.predict(X_transf)
    return y_hat

In [11]:
def test(D, algoritmo, ninter, inter_min, inter_max):
    dataset, algoritmo_l, msre_l = getResults('tests.pkl')
    pastas = ['0', '1', '2', '3', '4']
    dataset.append(D)
    algoritmo_l.append(f'{algoritmo} {ninter} ({inter_min} {inter_max})')
    msre_total = 0
    for pasta in pastas:
        fileTrain = 'datasets/' + D + '-train-' + pasta + '.dat'
        fileTest = 'datasets/' + D + '-test-' + pasta + '.dat'
        X_train, y_train = importaDados(fileTrain)
        X_test, y_test = importaDados(fileTest)
        n = X_train.shape[1]
        rede, modelo = fit(X_train, y_train, algoritmo, ninter*n, inter_min, inter_max+1)
        y_hat = predict(X_test, rede, modelo)
        msre_total += msre(y_test, y_hat)
    m = len(pastas)
    msre_medio = msre_total/m
    msre_l.append(msre_medio)
    storeResults(dataset, algoritmo_l, msre_l, 'tests.pkl')
    with open('tests.txt', 'a') as myfile:
        myfile.write(f'{D},{algoritmo} {ninter} ({inter_min} {inter_max}),{msre_medio}\n')
    print(f'{D},{algoritmo} {ninter} ({inter_min} {inter_max}),{msre_medio}')

In [ ]:
# faça esse somente para rodar todos os testes novamente
#datasets = ['airfoil', 'concrete', 'cpu', 'energyCooling', 'energyHeating', 'forestfires', 'towerData', 'wineRed', 'wineWhite', 'yacht']
#algoritmos = ['lassoLars', 'lassoLars', 'lassoLars', 'lassoLars', 'lassoLars', 'lasso', 'lassoLars', 'lassoLars', 'lassoLars', 'lassoLars']
#ninters = [10, 500, 1000, 500, 500, 10, 10, 100, 100, 100]
#inter_mins = [-2, -2, -3, -2, 0, -2, 0, -1, -1, -3]
#inter_maxs = [2, 2, 3, 2, 4, 2, 2, 1, 1, 3]
#for i in range(10):
#    test(datasets[i], algoritmos[i], ninters[i], inter_mins[i], inter_maxs[i])

In [16]:
# carregar os resultados dos testes do arquivo e gerar o dataframe de resultados
#dataset, algoritmo_l, msre_l = getResults('tests.pkl')
#df_IT = pd.DataFrame({'dataset':dataset, 'algoritmo':algoritmo_l, 'rmse':msre_l})
# salvar o df no arquivo
#fw = open('df_IT.pkl', 'wb')
#pickle.dump(df_IT, fw)
#fw.close()

In [17]:
# carregar o df do arquivo de resultados
fw = open('df_IT.pkl', 'rb')
df_IT = pickle.load(fw)
fw.close()

In [18]:
# carregar o df do arquivo de resultados dos outros algoritmos
fw = open('df_GS.pkl', 'rb')
df_GS = pickle.load(fw)
fw.close()

In [19]:
df = pd.concat([df_IT, df_GS], ignore_index=True)

In [20]:
datasets = df_IT.dataset.unique()
for D in datasets:
    print('\n', D)
    print(df.loc[df['dataset'] == D])


 airfoil
              algoritmo  dataset      rmse
0   lassoLars 10 (-2 2)  airfoil  2.831519
16                Ridge  airfoil  4.862221
17                Lasso  airfoil  6.142515
18                 Lars  airfoil  4.819300
19           ElasticNet  airfoil  6.169792
20              XGBoost  airfoil  2.186777
21                  MLP  airfoil  4.836487

 concrete
               algoritmo   dataset       rmse
1   lassoLars 500 (-2 2)  concrete   9.342848
40                 Ridge  concrete  10.444737
41                 Lasso  concrete  10.446343
42                  Lars  concrete  10.443756
43            ElasticNet  concrete  10.446860
44               XGBoost  concrete   4.758625
45                   MLP  concrete   7.686262

 cpu
                algoritmo dataset       rmse
2   lassoLars 1000 (-3 3)     cpu  21.160806
64                  Ridge     cpu  34.904959
65                  Lasso     cpu  42.059112
66                   Lars     cpu  34.546549
67             ElasticNet     cpu  4